In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import TCNModel
from darts.metrics import mse, rmse, r2_score, mae, smape
from darts.dataprocessing.transformers import Scaler
import torch
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import Callback, EarlyStopping
import optuna
import os
import json

# Visualization settings
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
torch.set_float32_matmul_precision('medium')

In [ ]:
target_columns = [
    'Temperature','Precipitation_accumulated','Humidity', 'Wind_Speed_kmh',
    'Soil_Moisture', 'Soil_Temperature', 'Wind_Dir_Sin', 'Wind_Dir_Cos'
]

DATA_FILE_PATH = "../data/ground_station_clean.csv"
df = pd.read_csv(DATA_FILE_PATH)

In [3]:
encoders = {
    "cyclic": {
        "past": ["month", "dayofyear", "day", "hour", "minute"],
        "future": ["month", "dayofyear", "day", "hour", "minute"]
    },
    "transformer": Scaler(),
    "datetime_attribute": {
        "past": ["year"],
        "future": ["year"]
    }
}


In [4]:
class PatchedPruningCallback(PyTorchLightningPruningCallback, Callback):
    pass

# Create directory to save iteration results
results_output_dir = "optuna_iteration_metrics"
os.makedirs(results_output_dir, exist_ok=True)

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

def objective(trial):
    input_chunk_length = trial.suggest_int("input_chunk_length", 24, 72)
    output_chunk_length = trial.suggest_int("output_chunk_length", 1, 24)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    batch_size = trial.suggest_int("batch_size", 64, 256)
    lr = trial.suggest_float("lr", 1e-6, 5e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)
    
    num_filters   = trial.suggest_int("num_filters", 32, 256, step=8)
    kernel_size   = trial.suggest_int("kernel_size", 2, 8)
    num_layers    = trial.suggest_int("num_layers", 0, 3)
    dilation_base = trial.suggest_int("dilation_base", 1, 4)

    num_filters   = trial.suggest_int("num_filters", 8, 128, log=True)
    kernel_size   = trial.suggest_int("kernel_size", 2, 8)
    num_layers    = trial.suggest_int("num_layers", 1, 3)
    dilation_base = trial.suggest_int("dilation_base", 1, 4)
    weight_norm = trial.suggest_categorical("weight_norm", [True, False])

    # Try different loss functions
    loss_functions = {
        'mse': torch.nn.MSELoss(),
        'mae': torch.nn.L1Loss(),
        'huber': torch.nn.HuberLoss(delta=1.0),
        'smoothl1': torch.nn.SmoothL1Loss()
    }
    loss_name = trial.suggest_categorical("loss_fn", list(loss_functions.keys()))

    # Try different optimizers
    optimizer_names = ['Adam', 'AdamW', 'RMSprop', 'SGD']
    optimizer_name = trial.suggest_categorical("optimizer", optimizer_names)
    optimizer_dict = {
        'Adam': torch.optim.Adam,
        'AdamW': torch.optim.AdamW,
        'RMSprop': torch.optim.RMSprop,
        'SGD': torch.optim.SGD
    }

    early_stopper = EarlyStopping("val_loss", min_delta=0.0005, patience=15, verbose=True)
    prunner = PatchedPruningCallback(trial, monitor="val_loss")
    pl_trainer_kwargs = {
        "accelerator": "auto",
        "callbacks": [early_stopper, prunner],
    }

    n_splits = 3
    tscv = TimeSeriesSplit(n_splits=n_splits)
    overall_smape_list = []
    metrics_per_fold = []
    fold = 0

    for train_index, val_index in tscv.split(df):
        print(f"\nFold {fold+1}/{n_splits}")
        train_df_fold = df.iloc[train_index]
        val_df_fold = df.iloc[val_index]
        train_fold = TimeSeries.from_dataframe(train_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')
        val_fold = TimeSeries.from_dataframe(val_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')

        scaler = Scaler()
        scaler = scaler.fit(train_fold)
        train_scaled = scaler.transform(train_fold)
        val_scaled = scaler.transform(val_fold)

        opt_kwargs = {"lr": lr, "weight_decay": weight_decay}

    model = TCNModel(
        model_name=_model_name,
        work_dir=_work_dir,
        input_chunk_length=input_chunk_length,
        output_chunk_length=output_chunk_length,
        num_filters=num_filters,
        kernel_size=kernel_size,
        num_layers=num_layers,
        dilation_base=dilation_base,
        weight_norm=weight_norm,
        n_epochs=30,
        batch_size=batch_size,
        dropout=dropout,
        add_encoders=encoders,
        pl_trainer_kwargs=pl_trainer_kwargs,
        loss_fn=loss_functions[loss_name],
        optimizer_cls=optimizer_dict[optimizer_name],
        lr_scheduler_cls=torch.optim.lr_scheduler.ReduceLROnPlateau,
        lr_scheduler_kwargs={"mode": "min", "factor": 0.3, "patience": 5, "min_lr": 1e-7},
        save_checkpoints=True,
        show_warnings=True,
        force_reset=True,
        optimizer_kwargs=opt_kwargs,
    )
    model.fit(
        series=train_scaled,
        val_series=val_scaled,
        verbose=False,
        dataloader_kwargs={"num_workers": 11},
    )

    try:
        loaded_model = TCNModel.load_from_checkpoint(model_name=_model_name, work_dir=_work_dir, best=True)
        print(f"Model loaded from checkpoint for trial {trial.number}")
    except FileNotFoundError:
        print(f"Checkpoint not found for {_model_name} in {_work_dir}. Using the in-memory trained model.")
        loaded_model = model

        forecasts = loaded_model.historical_forecasts(
            val_scaled,
            forecast_horizon=24,
            stride=1,
            retrain=False,
            verbose=False
        )


        forecasts_t = scaler.inverse_transform(forecasts)
        s = scaler.inverse_transform(val_scaled)

        overall_smape_val = smape(s, forecasts_t)
        print(f"SMAPE fold {fold}: {overall_smape_val}")
        overall_smape_list.append(overall_smape_val)
        
        metrics = {}
        for target in target_columns:
            metrics[target] = {
                'MSE': mse(s[target], forecasts_t[target]),
                'RMSE': rmse(s[target], forecasts_t[target]),
                'MAE': mae(s[target], forecasts_t[target]),
                'R2': r2_score(s[target], forecasts_t[target]),
                'SMAPE': smape(s[target], forecasts_t[target]),
                'WAPE': wmape(s[target], forecasts_t[target]),
            }
        metrics_per_fold.append(metrics)
        fold += 1

    mean_smape = np.mean(overall_smape_list)
    print(f"Média dos SMAPE nos folds: {mean_smape}")

    trial_dict = {
        "trial_number": trial.number,
        "input_chunk_length": input_chunk_length,
        "output_chunk_length": output_chunk_length,
        "num_filters": num_filters,
        "kernel_size": kernel_size,
        "num_layers": num_layers,
        "dilation_base": dilation_base,
        "weight_norm": weight_norm,
        "dropout": dropout,
        "batch_size": batch_size,
        "lr": lr,
        "loss_function": loss_name,
        "optimizer": optimizer_name,
        "overall_mae_val": overall_mae_val,
        "metrics": metrics_df.to_dict(),
    }
    json_path = os.path.join(results_output_dir, f"trial_{trial.number}.json")
    with open(json_path, 'w') as f:
        json.dump(trial_dict, f, indent=4)
    print(f"Results of trial {trial.number} saved in {json_path}")
    return mean_mse if not np.isnan(mean_mse) else float("inf")

In [ ]:
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [7]:
study = optuna.create_study(direction="minimize")
num_hyperparams = 13
n_trials = 7 * num_hyperparams
study.optimize(objective, n_trials=n_trials, callbacks=[print_callback])

[I 2025-05-25 20:54:41,027] A new study created in memory with name: no-name-d2e4f872-9d39-4afa-a3d9-c671d5f771b9



Starting Trial 0
Hyperparameters: {'input_chunk_length': 42, 'output_chunk_length': 19, 'dropout': 0.10970982839711962, 'batch_size': 256, 'lr': 3.835126443855208e-05, 'weight_decay': 1.4794457276648664e-06, 'num_filters': 29, 'kernel_size': 8, 'num_layers': 1, 'dilation_base': 2, 'weight_norm': False, 'loss_fn': 'mse', 'optimizer': 'AdamW'}

Training the model...
Train set: 6647 samples
Validation set: 1662 samples


Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Metric val_loss improved. New best score: 0.629
Metric val_loss improved by 0.084 >= min_delta = 0.0005. New best score: 0.545
Metric val_loss improved by 0.081 >= min_delta = 0.0005. New best score: 0.465
Metric val_loss improved by 0.076 >= min_delta = 0.0005. New best score: 0.388
Metric val_loss improved by 0.067 >= min_delta = 0.0005. New best score: 0.322
Metric val_loss improved by 0.051 >= min_delta = 0.0005. New best score: 0.270
Metric val_loss improved by 0.035 >= min_delta = 0.0005. New best score: 0.235
Metric val_loss improved by 0.022 >= min_delta = 0.0005. New best score: 0.213
Metric val_loss improved by 0.013 >= min_delta = 0.0005. New best score: 0.201
Metric val_loss improv

Model loaded from checkpoint for trial 0


/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
[I 2025-05-25 20:55:23,552] Trial 0 finished with value: 0.25685879562761493 and parameters: {'input_chunk_length': 42, 'output_chunk_length': 19, 'dropout': 0.10970982839711962, 'batch_size': 256, 'lr': 3.835126443855208e-05, 'weight_decay': 1.4794457276648664e-06, 'num_filters': 29, 'kernel_size': 8, 'num_layers': 1, 'dilation_base': 2, 'weight_norm': False, 'loss_fn': 'mse', 'optimizer': 'AdamW'}. Best is trial 0 with value: 0.25685879562761493.
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.
GPU available: True (cuda), used: True
TPU available: Fal

...

Current value: 0.02575839519271729, Current params: {'input_chunk_length': 30, 'output_chunk_length': 1, 'dropout': 0.10936898993536755, 'batch_size': 107, 'lr': 0.00425020904789375, 'weight_decay': 0.00016390798400110304, 'num_filters': 44, 'kernel_size': 8, 'num_layers': 3, 'dilation_base': 4, 'weight_norm': True, 'loss_fn': 'smoothl1', 'optimizer': 'Adam'}
Best value: 0.08353507711014438, Best params: {'input_chunk_length': 29, 'output_chunk_length': 1, 'dropout': 0.10987578257441336, 'batch_size': 107, 'lr': 0.004525867812805064, 'weight_decay': 0.00015619537785532893, 'num_filters': 50, 'kernel_size': 8, 'num_layers': 3, 'dilation_base': 4, 'weight_norm': True, 'loss_fn': 'smoothl1', 'optimizer': 'AdamW'}

Starting Trial 90
Hyperparameters: {'input_chunk_length': 32, 'output_chunk_length': 1, 'dropout': 0.12363452166863391, 'batch_size': 86, 'lr': 0.004065715349889218, 'weight_decay': 5.639205865576639e-05, 'num_filters': 50, 'kernel_size': 8, 'num_layers': 3, 'dilation_base': 2, 

Metric val_loss improved. New best score: 0.021
Metric val_loss improved by 0.005 >= min_delta = 0.0005. New best score: 0.016
Metric val_loss improved by 0.002 >= min_delta = 0.0005. New best score: 0.014
Metric val_loss improved by 0.001 >= min_delta = 0.0005. New best score: 0.014
Metric val_loss improved by 0.001 >= min_delta = 0.0005. New best score: 0.013
Metric val_loss improved by 0.001 >= min_delta = 0.0005. New best score: 0.012
Metric val_loss improved by 0.001 >= min_delta = 0.0005. New best score: 0.012
Monitored metric val_loss = nan is not finite. Previous best value was 0.012. Signaling Trainer to stop.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Model loaded from checkpoint for trial 90


/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
[I 2025-05-25 23:59:07,394] Trial 90 finished with value: 0.08570353493707669 and parameters: {'input_chunk_length': 32, 'output_chunk_length': 1, 'dropout': 0.12363452166863391, 'batch_size': 86, 'lr': 0.004065715349889218, 'weight_decay': 5.639205865576639e-05, 'num_filters': 50, 'kernel_size': 8, 'num_layers': 3, 'dilation_base': 2, 'weight_norm': True, 'loss_fn': 'smoothl1', 'optimizer': 'Adam'}. Best is trial 83 with value: 0.08353507711014438.


Starting time series verification for consistency...

Performance metrics:
                                    MSE        RMSE         MAE         R2  \
Temperature                    2.029034    1.424442    1.165297   0.933414   
Precipitation_accumulated     82.118358    9.061918    7.349219 -23.038320   
Humidity                      10.415943    3.227374    2.510384   0.928905   
Wind_Speed_kmh                 0.535453    0.731746    0.566032   0.602680   
Soil_Moisture              14196.502169  119.149075  103.490213   0.748977   
Soil_Temperature               3.046297    1.745365    1.527083   0.810734   
Wind_Dir_Sin                   0.374140    0.611670    0.502901   0.320510   
Wind_Dir_Cos                   0.315196    0.561423    0.458994   0.139982   

                                SMAPE  
Temperature                 23.791222  
Precipitation_accumulated    1.595483  
Humidity                     2.960728  
Wind_Speed_kmh              57.900963  
Soil_Moisture         

In [8]:
print(f"Best MASE Value (Minimum): {study.best_value}")
print(f"Best Parameters: {study.best_params}")

Best Value (Minimum): 0.08353507711014438
Best Parameters: {'input_chunk_length': 29, 'output_chunk_length': 1, 'dropout': 0.10987578257441336, 'batch_size': 107, 'lr': 0.004525867812805064, 'weight_decay': 0.00015619537785532893, 'num_filters': 50, 'kernel_size': 8, 'num_layers': 3, 'dilation_base': 4, 'weight_norm': True, 'loss_fn': 'smoothl1', 'optimizer': 'AdamW'}


In [ ]:
best_dict = {
    "best_value": study.best_value,
    "best_params": study.best_params,
}
json_path = os.path.join(results_output_dir, "best_trial.json")
with open(json_path, 'w') as f:
    json.dump(best_dict, f, indent=4)
print(f"Best results saved in {json_path}")


Best results saved in optuna_iteration_metrics/best_trial.json
